## How to use the UniRep mLSTM "babbler". This version demonstrates the 64-unit and the 1900-unit architecture. 

We recommend getting started with the 64-unit architecture as it is easier and faster to run, but has the same interface as the 1900-unit one.

Use the 64-unit or the 1900-unit model?

In [1]:
USE_FULL_1900_DIM_MODEL = False # if True use 1900 dimensional model, else use 64 dimensional one.

## Setup

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

# Set seeds
tf.set_random_seed(42)
np.random.seed(42)

if USE_FULL_1900_DIM_MODEL:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/1900_weights/ 1900_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler1900 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./1900_weights"
    
else:
    # Sync relevant weight files
    !aws s3 sync --no-sign-request --quiet s3://unirep-public/64_weights/ 64_weights/
    
    # Import the mLSTM babbler model
    from unirep import babbler64 as babbler
    
    # Where model weights are stored.
    MODEL_WEIGHT_PATH = "./64_weights"

## Data formatting and management

Initialize UniRep, also referred to as the "babbler" in our code. You need to provide the batch size you will use and the path to the weight directory.

In [3]:
batch_size = 63
b = babbler(batch_size=batch_size, model_path=MODEL_WEIGHT_PATH)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


UniRep needs to receive data in the correct format, a (batch_size, max_seq_len) matrix with integer values, where the integers correspond to an amino acid label at that position, and the end of the sequence is padded with 0s until the max sequence length to form a non-ragged rectangular matrix. We provide a formatting function to translate a string of amino acids into a list of integers with the correct codex:

In [4]:
seq_open = open("./adimab_seq.txt")
seq = seq_open.read()
print(seq)

QVQLVESGGGVVQPGRSLRLSCAASGFTFSSYGMHWVRQAPGKGLEWVAFIWYEGSNKYYAESVKDRFTISRDNSKNTLYLQMKSLRAEDTAVYYCARRAGIIGTIGYYYGMDVWGQGTTVTVSSSYELTQPPSVSVSPGQTASITCSGDRLGEKYTSWYQQRPGQSPLLVIYQDTKRPSGIPERFSGSNSGNTATLTISGTQAMDEADYYCQAWESSTVVFGGGTKLTVL
EVQLVQSGGGVERPGGSLRLSCAASGFTFDDYAMSWVRQAPGKGLEWVSGINWQGGSTGYADSVKGRVTISRDNAKNSLYLQMNSLRAEDTAVYYCAKILGAGRGWYFDYWGKGTTVTVSSSELTQDPAVSVALGQTVRITCSGDSLRSYYASWYQQKPGQAPVLVIYGANNRPSGIPDRFSGSSSGNTASLTITGAQAEDEADYYCNSADSSGNHVVFGGGTKLTVL
QVQLVESGGGVVQPGGSLRLSCAASGFTFSSYGMHWVRQAPGKGLEWVAFIWYEGSNKYYAESVKDRFTISRDNSKNTLYLQMNSLRAEDTAVYYCARRAGIIGTIGYYYGMDVWGQGTTVTVSSSYELTQPPSVSVSPGQTASITCSGDRLGEKYTSWYQQRPGQSPLLVIYQDTKRPSGIPERFSGSNSGNTATLTISGTQAMDEADYYCQAWESSTVVFGGGTKLTVL
QVQLVESGGGVVQPGRSLRLSCAASGFTFSSYGMHWVRQAPGKGLEWVAVIWYSGSNKYYADAVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCARDRLTTYPTYYGMDVWGQGTTVTVSSQSALTQPASVSGSPGQSITISCTGTSSAVGNYNLVSWYQQHPGKAPKLMISEVSKRPSGVSHRFSGSKSGNTASLTISGLQAEDEADYYCSSYAGSNNLMFGGGTKLTVL
QVQLVESGGGVVQPGRSLRLSCAASGFTFSSYGMHWVRQAPGKGLEWVAVIWYDASNKYYADSVKGRFTISRD

In [5]:
seq_formatted = []
for line in seq.splitlines():
    seq_formatted.append(np.array(b.format_seq(line)))
print(seq_formatted)

[array([24, 10, 16, 10, 21, 16,  6,  7, 13, 13, 13, 16, 16, 10, 14, 13,  2,
        7, 21,  2, 21,  7, 11, 15, 15,  7, 13, 18,  8, 18,  7,  7, 19, 13,
        1,  3, 20, 16,  2, 10, 15, 14, 13,  4, 13, 21,  6, 20, 16, 15, 18,
       17, 20, 19,  6, 13,  7,  9,  4, 19, 19, 15,  6,  7, 16,  4,  5,  2,
       18,  8, 17,  7,  2,  5,  9,  7,  4,  9,  8, 21, 19, 21, 10,  1,  4,
        7, 21,  2, 15,  6,  5,  8, 15, 16, 19, 19, 11, 15,  2,  2, 15, 13,
       17, 17, 13,  8, 17, 13, 19, 19, 19, 13,  1,  5, 16, 20, 13, 10, 13,
        8,  8, 16,  8, 16,  7,  7,  7, 19,  6, 21,  8, 10, 14, 14,  7, 16,
        7, 16,  7, 14, 13, 10,  8, 15,  7, 17,  8, 11,  7, 13,  5,  2, 21,
       13,  6,  4, 19,  8,  7, 20, 19, 10, 10,  2, 14, 13, 10,  7, 14, 21,
       21, 16, 17, 19, 10,  5,  8,  4,  2, 14,  7, 13, 17, 14,  6,  2, 18,
        7, 13,  7,  9,  7, 13,  9,  8, 15,  8, 21,  8, 17,  7, 13,  8, 10,
       15,  1,  5,  6, 15,  5, 19, 19, 11, 10, 15, 20,  6,  7,  7,  8, 16,
       16, 18, 13, 13, 1

We also provide a function that will check your amino acid sequences don't contain any characters which will break the UniRep model.

In [6]:
b.is_valid_seq(seq)

False

You could use your own data flow as long as you ensure that the data format is obeyed. Alternatively, you can use the data flow we've implemented for UniRep training, which happens in the tensorflow graph. It reads from a file of integer sequences, shuffles them around, collects them into groups of similar length (to minimize padding waste) and pads them to the max_length. Here's how to do that:

First, sequences need to be saved in the correct format. Suppose we have a new-line seperated file of amino acid sequences, `seqs.txt`, and we want to format them. Note that training is currently only publicly supported for amino acid sequences less than 275 amino acids as gradient updates for sequences longer than that start to get unwieldy. If you want to train on sequences longer than this, please reach out to us. 

Sequence formatting can be done as follows:

In [7]:
# Before you can train your model, 
with open("adimab_seq.txt", "r") as source:
    with open("formatted.txt", "w") as destination:
        for i,seq in enumerate(source):
            seq = seq.strip()
            if b.is_valid_seq(seq) and len(seq) < 275: 
                formatted = ",".join(map(str,b.format_seq(seq)))
                destination.write(formatted)
                destination.write('\n')


This is what the integer format looks like

In [8]:
!head -n1 formatted.txt

24,10,16,10,21,16,6,7,13,13,13,16,16,10,14,13,2,7,21,2,21,7,11,15,15,7,13,18,8,18,7,7,19,13,1,3,20,16,2,10,15,14,13,4,13,21,6,20,16,15,18,17,20,19,6,13,7,9,4,19,19,15,6,7,16,4,5,2,18,8,17,7,2,5,9,7,4,9,8,21,19,21,10,1,4,7,21,2,15,6,5,8,15,16,19,19,11,15,2,2,15,13,17,17,13,8,17,13,19,19,19,13,1,5,16,20,13,10,13,8,8,16,8,16,7,7,7,19,6,21,8,10,14,14,7,16,7,16,7,14,13,10,8,15,7,17,8,11,7,13,5,2,21,13,6,4,19,8,7,20,19,10,10,2,14,13,10,7,14,21,21,16,17,19,10,5,8,4,2,14,7,13,17,14,6,2,18,7,13,7,9,7,13,9,8,15,8,21,8,17,7,13,8,10,15,1,5,6,15,5,19,19,11,10,15,20,6,7,7,8,16,16,18,13,13,13,8,4,21,8,16,21


Notice that by default format_seq does not include the stop symbol (25) at the end of the sequence. This is the correct behavior if you are trying to train a top model, but not if you are training UniRep representations.

Now we can use a custom function to bucket, batch and pad sequences from `formatted.txt` (which has the correct integer codex after calling `babbler.format_seq()`). The bucketing occurs in the graph. 

What is bucketing? Specify a lower and upper bound, and interval. All sequences less than lower or greater than upper will be batched together. The interval defines the "sides" of buckets between these bounds. Don't pick a small interval for a small dataset because the function will just repeat a sequence if there are not enough to
fill a batch. All batches are the size you passed when initializing the babbler.

This is also doing a few other things:
- Shuffling the sequences by randomly sampling from a 10000 sequence buffer
- Automatically padding the sequences with zeros so the returned batch is a perfect rectangle
- Automatically repeating the dataset

In [9]:
bucket_op = b.bucket_batch_pad("formatted.txt", interval=1000) # Large interval

Inconveniently, this does not make it easy for a value to be associated with each sequence and not lost during shuffling. You can get around this by just prepending every integer sequence with the sequence label (eg, every sequence would be saved to the file as "{brightness value}, 24, 1, 5,..." and then you could just index out the first column after calling the `bucket_op`. Please reach out if you have questions on how to do this.

Now that we have the `bucket_op`, we can simply `sess.run()` it to get a correctly formatted batch

In [10]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    batch = sess.run(bucket_op)
    
print(batch)
print(batch.shape)

[[24 10 16 ...  0  0  0]
 [24 10 16 ...  0  0  0]
 [24 10 16 ...  0  0  0]
 ...
 [24 10 16 ...  0  0  0]
 [24  6 16 ...  0  0  0]
 [24 10 16 ...  0  0  0]]
(63, 242)


You can look back and see that the batch_size we passed to __init__ is indeed 12, and the second dimension must be the longest sequence included in this batch. Now we have the data flow setup (note that as long as your batch looks like this, you don't need my flow), so we can proceed to implementing the graph. The module returns all the operations needed to feed in sequence and get out trainable representations.

## Training a top model and a top model + mLSTM.

First, obtain all of the ops needed to output a representation

In [11]:
final_hidden, x_placeholder, batch_size_placeholder, seq_length_placeholder, initial_state_placeholder = (
    b.get_rep_ops())

`final_hidden` should be a batch_size x rep_dim matrix.

Lets say we want to train a basic feed-forward network as the top model, doing regression with MSE loss, and the Adam optimizer. We can do that by:

1.  Defining a loss function.

2.  Defining an optimizer that's only optimizing variables in the top model.

3.  Minimizing the loss inside of a TensorFlow session

In [12]:
y_placeholder = tf.placeholder(tf.float32, shape=[None,1], name="y")
initializer = tf.contrib.layers.xavier_initializer(uniform=False)

with tf.variable_scope("top"):
    prediction = tf.contrib.layers.fully_connected(
        final_hidden, 1, activation_fn=None, 
        weights_initializer=initializer,
        biases_initializer=tf.zeros_initializer()
    )

loss = tf.losses.mean_squared_error(y_placeholder, prediction)

You can specifically train the top model first by isolating variables of the "top" scope, and forcing the optimizer to only optimize these.

In [13]:
learning_rate=.002
top_variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="top")
optimizer = tf.train.AdamOptimizer(learning_rate)
top_only_step_op = optimizer.minimize(loss, var_list=top_variables)
all_step_op = optimizer.minimize(loss)

We next need to define a function that allows us to calculate the length each sequence in the batch so that we know what index to use to obtain the right "final" hidden state

In [14]:
def nonpad_len(batch):
    nonzero = batch > 0
    lengths = np.sum(nonzero, axis=1)
    return lengths

nonpad_len(batch)

array([227, 229, 237, 225, 226, 235, 232, 229, 232, 229, 227, 230, 242,
       229, 229, 237, 229, 226, 234, 232, 235, 237, 230, 232, 226, 231,
       224, 234, 224, 240, 225, 228, 231, 230, 226, 230, 229, 231, 233,
       228, 228, 231, 231, 229, 228, 232, 229, 229, 228, 231, 226, 227,
       227, 233, 229, 228, 230, 234, 226, 232, 224, 229, 228])

We are ready to train. As an illustration, let's learn to predict the number 42 just optimizing the top model.

In [41]:
#visc_open = open("./adimab_visc.txt")
y = [[16.8], [348.0], [22.7], [6.0], [8.1], [6.1], [8.9], [11.9], [11.7], [19.7], [180.0], [14.6], [12.6], [9.9], [8.6], [10.3], [26.4], [10.8], [45.0], [27.0], [12.4], [219.1], [11.9], [45.0], [11.4], [8.9], [9.3], [14.6], [10.3], [12.6], [12.2], [9.4], [21.8], [8.4], [17.8], [10.4], [12], [15.3], [15.2], [40.0], [10.9], [48.8], [12.0], [15.0], [13.1], [5.5], [10.8], [13.9], [21.7], [31.8], [16.6], [17.1], [7.6], [14.8], [21.2], [13.9], [45.9], [27.3], [13.8], [25.6], [11.7], [12.1], [17.6]]
num_iters = 10
predictions = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(num_iters):
        batch = sess.run(bucket_op)
        length = nonpad_len(batch)
        loss_, __, = sess.run([loss, top_only_step_op],
                feed_dict={
                     x_placeholder: batch,
                     y_placeholder: y,
                     batch_size_placeholder: batch_size,
                     seq_length_placeholder:length,
                     initial_state_placeholder:b._zero_state
                }
        )
        
        print("Iteration {0}: {1}".format(i, loss_))

Iteration 0: 3550.65673828125
Iteration 1: 3548.91796875
Iteration 2: 3547.181396484375
Iteration 3: 3545.44677734375
Iteration 4: 3543.71484375
Iteration 5: 3541.983642578125
Iteration 6: 3540.256103515625
Iteration 7: 3538.531005859375
Iteration 8: 3536.80615234375
Iteration 9: 3535.085205078125


We can also jointly train the top model and the mLSTM. Note that if using the 1900-unit (full) model, you will need a GPU with at least 16GB RAM. To see a demonstration of joint training with fewer computational resources, please run this notebook using the 64-unit model.

In [56]:
y = [[16.8], [348.0], [22.7], [6.0], [8.1], [6.1], [8.9], [11.9], [11.7], [19.7], [180.0], [14.6], [12.6], [9.9], [8.6], [10.3], [26.4], [10.8], [45.0], [27.0], [12.4], [219.1], [11.9], [45.0], [11.4], [8.9], [9.3], [14.6], [10.3], [12.6], [12.2], [9.4], [21.8], [8.4], [17.8], [10.4], [12], [15.3], [15.2], [40.0], [10.9], [48.8], [12.0], [15.0], [13.1], [5.5], [10.8], [13.9], [21.7], [31.8], [16.6], [17.1], [7.6], [14.8], [21.2], [13.9], [45.9], [27.3], [13.8], [25.6], [11.7], [12.1], [17.6]]
print(y)
print(batch)
num_iters = 2
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(num_iters):
        batch = sess.run(bucket_op)
        length = nonpad_len(batch)
        loss_, __, = sess.run([loss, all_step_op],
                feed_dict={
                     x_placeholder: batch,
                     y_placeholder: y,
                     batch_size_placeholder: batch_size,
                     seq_length_placeholder:length,
                     initial_state_placeholder:b._zero_state
                }
        )
        prediction = tf.argmax(y,1)
        print(prediction.eval(feed_dict={x_placeholder: batch}))
        print(prediction)
        print("Iteration {0}: {1}".format(i,loss_))

[[16.8], [348.0], [22.7], [6.0], [8.1], [6.1], [8.9], [11.9], [11.7], [19.7], [180.0], [14.6], [12.6], [9.9], [8.6], [10.3], [26.4], [10.8], [45.0], [27.0], [12.4], [219.1], [11.9], [45.0], [11.4], [8.9], [9.3], [14.6], [10.3], [12.6], [12.2], [9.4], [21.8], [8.4], [17.8], [10.4], [12], [15.3], [15.2], [40.0], [10.9], [48.8], [12.0], [15.0], [13.1], [5.5], [10.8], [13.9], [21.7], [31.8], [16.6], [17.1], [7.6], [14.8], [21.2], [13.9], [45.9], [27.3], [13.8], [25.6], [11.7], [12.1], [17.6]]
[[24 10 16 ...  0  0  0]
 [24 10 16 ...  0  0  0]
 [24 10 16 ...  0  0  0]
 ...
 [24 10 16 ...  0  0  0]
 [24  6 16 ...  0  0  0]
 [24 10 16 ...  0  0  0]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Tensor("ArgMax_123:0", shape=(63,), dtype=int64)
Iteration 0: 3550.65673828125
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Tensor("ArgMa

[<tf.Tensor 'top/fully_connected/BiasAdd:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'top/fully_connected/BiasAdd:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'top/fully_connected/BiasAdd:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'top/fully_connected/BiasAdd:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'top/fully_connected/BiasAdd:0' shape=(?, 1) dtype=float32>]
